# Word Matcher - Applicazione non biologica

Di seguito è mostrata l'applicazione dell'allineamento di sequenza alla ricerca della parola più simile a una stringa data.

Importiamo il modulo amplpy per risolvere i problemi di programmazione intera

In [1]:
from amplpy import AMPL, add_to_path
add_to_path(r"C:\ampl")

Importiamo il modulo nltk per ottenere una lista di parole inglesi

In [2]:
import nltk
nltk.download('words')
from nltk.corpus import words

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


Funzione di utilità che scrive il file di dati che verrà utilizzato da ampl

In [3]:
def writeData(s1, s2, filename = "ampl_files/input.dat"):
    
    file = open(filename, "w")
    file.writelines([
            f"param n:={len(s1)};\n",
            f"param m:={len(s2)};\n",
            "param gap_p:=1;\n",
            "param mis_p:=1;\n",
            "param r:=-1;\n",
            "param a :=\n"
        ])
    
    a = []
    for c1 in s1:
      row = []
      for c2 in s2:
        row.append(1 if c1 == c2 else 0)
      a.append(row)
    
    for i in range(len(a)):
      for j in range(len(a[i])):
        file.write(f"{i+1} {j+1} {a[i][j]} \n")
    
    file.write(";\n")
    file.close()

La seguente funzione filtra le parole che differiscono eccessivamente dalla stringa di input. Vengono eliminate le parole che non condividono alcuna lettera con la stringa obiettivo e quelle con una lunghezza significativamente diversa.

In [4]:
def tooDifferent(a, b):
    if abs(len(a)-len(b)) > 0.5*len(a):
        return True
    for c in a:
        if c in b:
            return False
    return True

La funzione ```computeScore``` applica il modello dell'allineamento di sequenza per ottenere il punteggio di somiglianza tra due parole.

In [5]:
def computeScore(target, word, inputfile):
    ampl = AMPL()
    best = -100, ""
    writeData(target, word, inputfile)

    ampl.eval(r''' 
        reset;
        model ./ampl_files/SequenceAlignment.mod;
        data ''' + inputfile + ''';
        option solver cbc;
        option solver_msg 0; 
        solve >ampl_files/out.txt;
        ''')

    score = -round(ampl.get_value("obj"))
    return score

In [ ]:
from tqdm import tqdm

english_words = words.words()
text = "telo"

if text not in english_words:

    filtered = list(filter(lambda w: not tooDifferent(text, w), english_words))
    n=len(filtered)

    best = -100, ""
    for i in tqdm(range(0, n), colour="#00ffff", desc ="Progress: "):
        score = computeScore(text, filtered[i], "ampl_files/word_input.dat")
        if score > best[0]:
            best = score, filtered[i]
            if score >= len(text)-2:
                break
    print(f"Best match: {best}")
else:
    print(f"{text} is a correct word")


Progress:   0%|▏                                                                 | 104/28636 [00:33<2:30:09,  3.17it/s]